
# Deploy Identifikasi Suara Buka dan    Tutup

Notebook ini menjelaskan **langkah-langkah lengkap pembuatan dan deployment aplikasi deteksi suara buka/tutup** berbasis *machine learning* dan **Streamlit**.

---

## Struktur Folder Proyek

Struktur folder proyek yang digunakan:

```
deploy_bukatutup/
│
├── data/
│   └── bukatutup.csv
│
├── train_model.py
├── app.py
├── model_bukatutup.pkl
├── label_encoder.pkl
├── requirements.txt
└── deploy_bukatutup.ipynb
```

---

##  File `requirements.txt`

File ini berisi daftar pustaka Python yang diperlukan agar aplikasi berjalan di lingkungan mana pun.

```text
streamlit
pandas
numpy
scikit-learn
librosa
joblib
```

**Penjelasan:**
- `streamlit` → framework untuk membuat aplikasi web interaktif.
- `pandas`, `numpy` → pengolahan data dan numerik.
- `scikit-learn` → algoritma machine learning.
- `librosa` → ekstraksi fitur dari sinyal audio.
- `joblib` → untuk menyimpan dan memuat model.

---

## File `train_model.py`

Script ini digunakan untuk **melatih model klasifikasi suara buka/tutup** menggunakan fitur statistik audio.

```python
# ==========================================
# train_model.py
# ==========================================
# Melatih model klasifikasi suara buka/tutup
# berdasarkan fitur statistik audio (mean, std, dll)
# ==========================================

import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

print("🚀 Mulai proses training model...")

# 1️⃣ Baca dataset
df = pd.read_csv("data/bukatutup.csv")
print(f"✅ Dataset terbaca dengan {df.shape[0]} baris dan kolom: {list(df.columns)}")

# 2️⃣ Deteksi kolom label
label_col = None
for col in ["label", "class", "kategori", "target"]:
    if col in df.columns:
        label_col = col
        break
if label_col is None:
    raise ValueError("❌ Kolom label tidak ditemukan! Harus ada kolom 'class' atau 'label'.")

print(f"🧩 Kolom label terdeteksi: '{label_col}'")

# 3️⃣ Hapus kolom non-numerik
if "file_name" in df.columns:
    df = df.drop(columns=["file_name"])
    print("🧹 Kolom 'file_name' dihapus karena non-numerik.")

# 4️⃣ Pisahkan fitur dan label
X = df.drop(columns=[label_col])
y = df[label_col]

# 5️⃣ Encoding label
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
print(f"🔢 Label unik: {list(encoder.classes_)}")

# 6️⃣ Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# 7️⃣ Training model
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# 8️⃣ Evaluasi
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"✅ Akurasi Model: {acc:.4f}")
print("\n📊 Laporan Klasifikasi:\n", classification_report(y_test, y_pred, target_names=encoder.classes_))

# 9️⃣ Simpan model & encoder
joblib.dump(model, "model_bukatutup.pkl")
joblib.dump(encoder, "label_encoder.pkl")
print("💾 Model dan encoder berhasil disimpan!")

print("🎉 Training selesai!")
```

**Penjelasan Langkah Utama:**
1. Membaca data dari folder `data/`.
2. Mendeteksi kolom label (class).
3. Menghapus kolom non-numerik (`file_name`).
4. Memisahkan fitur (`X`) dan label (`y`).
5. Melakukan encoding label teks menjadi angka.
6. Melatih model `RandomForestClassifier`.
7. Mengevaluasi model dan menampilkan akurasi.
8. Menyimpan model dan encoder ke file `.pkl`.

---

##  File `app.py`

Aplikasi **Streamlit** untuk mendeteksi jenis suara dari file audio `.wav` atau file fitur `.csv`.

```python
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import librosa

# Load model & encoder
model = joblib.load("model_bukatutup.pkl")
encoder = joblib.load("label_encoder.pkl")

st.title("🎙️ Identifikasi Suara Buka/Tutup")
st.write("Aplikasi ini mendeteksi jenis suara berdasarkan fitur statistik audio.")

option = st.radio("Pilih input data:", ["Upload fitur (CSV)", "Upload audio (.wav)"])

if option == "Upload fitur (CSV)":
    uploaded_file = st.file_uploader("Unggah file CSV berisi fitur statistik", type=["csv"])
    if uploaded_file is not None:
        df = pd.read_csv(uploaded_file)
        if "file_name" in df.columns:
            df = df.drop(columns=["file_name"])
        if "class" in df.columns:
            df = df.drop(columns=["class"])
        pred = model.predict(df)
        label_pred = encoder.inverse_transform(pred)
        st.success("✅ Prediksi berhasil!")
        st.write(pd.DataFrame({"Prediksi": label_pred}))

elif option == "Upload audio (.wav)":
    audio_file = st.file_uploader("Unggah file suara (format .wav)", type=["wav"])
    if audio_file is not None:
        st.audio(audio_file, format='audio/wav')
        y, sr = librosa.load(audio_file, sr=None)

        zcr = librosa.feature.zero_crossing_rate(y)[0]
        rmse = librosa.feature.rms(y=y)[0]
        centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

        features = {
            "zcr_mean": np.mean(zcr),
            "zcr_std": np.std(zcr),
            "rmse_mean": np.mean(rmse),
            "rmse_std": np.std(rmse),
            "centroid_mean": np.mean(centroid),
            "centroid_std": np.std(centroid),
            "bandwidth_mean": np.mean(bandwidth),
            "bandwidth_std": np.std(bandwidth),
            "rolloff_mean": np.mean(rolloff),
            "rolloff_std": np.std(rolloff),
        }
        for i in range(13):
            features[f"mfcc{i+1}_mean"] = np.mean(mfcc[i])
            features[f"mfcc{i+1}_std"] = np.std(mfcc[i])

        df_feat = pd.DataFrame([features])
        pred = model.predict(df_feat)
        label_pred = encoder.inverse_transform(pred)[0]
        st.success(f"🔊 Hasil Prediksi: **{label_pred.upper()}**")
```

**Penjelasan:**
- `streamlit` digunakan untuk antarmuka interaktif.
- Opsi input: 
  - **CSV** (fitur yang sudah diekstraksi).
  - **Audio WAV** (dievaluasi langsung oleh `librosa`).
- Menghasilkan prediksi dengan menampilkan label hasil model.

---

## Cara Menjalankan Aplikasi

1. Pastikan semua file sudah ada di folder.
2. Jalankan perintah berikut di terminal:

```bash
pip install -r requirements.txt
```

3. Jalankan aplikasi Streamlit:

```bash
streamlit run app.py
```

4. Aplikasi akan terbuka di browser (biasanya di `http://localhost:8501`).

---

##  Hasil dan Pengujian

Kamu bisa mencoba mengunggah file audio (contoh: `buka.wav`, `tutup.wav`) atau file CSV berisi fitur.  
Model akan menampilkan hasil klasifikasi: **Buka** atau **Tutup**.


